In [1]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from html import unescape
from typing import List, Dict

In [2]:
def get_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

In [3]:
def get_table(soup):
    tables = soup.find_all("table")
    return tables[0]

In [4]:
def convert(index: List[str]) -> List[int]:
    return list(map(int, index))

In [5]:
def parse_table(table) -> List[Dict]:
    trs = table.find_all("tr")

    def get_content(tr):
        ths = tr.find_all("th")
        if not ths:
            ths = tr.find_all("td")
        return [unescape(th.text).replace("\xa0", "") for th in ths]

    def find_min_max(items: List[str]) -> List[Dict]:
        result = list()
        for item in items:
            index1 = item[2:4]
            result.append({"valore_mercato": convert(index1)})
        return result

    items = [get_content(tr) for tr in trs]
    items = list(filter(lambda x: len(x) == 8, items))
    return find_min_max(items)

In [6]:
def generate_min_max(tabl):
    ll, v = [], set()
    for d in tabl:
        k = d["valore_mercato"]
        ll.append(k)
    df = pd.DataFrame(ll).rename(columns={0: 'min', 1: 'max'}).mean()
    data = pd.DataFrame(df).transpose()
    return data

In [7]:
def scrape_values(urls):
  dataframe = pd.DataFrame(columns=['min','max'])
  data_broken = [['Broken', 'Broken']]
  dt_broken = pd.DataFrame(data_broken, columns=['min','max'])
  for url in urls:
    try:
      page = get_page(url)
      parsed_tab = parse_table(get_table(page))
      row = generate_min_max(parsed_tab)
      dataframe = pd.concat([dataframe, row], axis=0)
      time.sleep(1)
    except:
      dataframe = pd.concat([dataframe, dt_broken])
      continue
  dataframe['ref_urls'] = urls
  dataframe.reset_index(drop=True, inplace=True)
  print('Dataset succesfully created:')
  display(dataframe.head())
  return dataframe

Proviamo il bs4 scraper

In [8]:
URL1 = "https://www1.agenziaentrate.gov.it/servizi/geopoi_omi/stampa.php?id=2157&pr=CE&co=B963&linkzona=CE00003358&idstrada=&anno_semestre=20221&fasciazona=Centrale/CENTRO%20URBANO%20A%20SUD%20DEL%20CENTRO%20STORICO&utilizzo=Commerciale&codzona=B5&lingua=IT&bt1=Mostra%20valori&E=14.334402&N=41.068798"
URL2 = "https://www1.agenziaentrate.gov.it/servizi/geopoi_omi/stampa.php?id=2157&pr=TP&co=F061&linkzona=TP00001360&idstrada=&anno_semestre=20221&fasciazona=Semicentrale/ZONA%20ANULARE%20ESTERNA%20AL%20CENTRO%20URBANO&utilizzo=Commerciale&codzona=C1&lingua=IT&bt1=Mostra%20valori&E=12.594715&N=37.649745"
URL3 = "https://www1.agenziaentrate.gov.it/servizi/geopoi_omi/stampa.php?id=2157&pr=AP&co=D542&linkzona=AP00000351&idstrada=&anno_semestre=20221&fasciazona=Semicentrale/FASCIA%20PERIMETRALE%20INTORNO%20AL%20CENTRO%20STORICO&utilizzo=Commerciale&codzona=C1&lingua=IT&bt1=Mostra%20valori&E=13.722029&N=43.161035"
URL4 = "https://www.google.it/"
URL5 = "blanksnekkecot34o"

In [9]:
prova = []
prova.append(URL1)
prova.append(URL2)
prova.append(URL5)
prova.append(URL3)
prova.append(URL4)

In [10]:
datafra = scrape_values(prova)

Dataset succesfully created:


,min,max,ref_urls
0,1550.0,3100.0,https://www1.agenziaentrate.gov.it/servizi/geo...
1,700.0,1050.0,https://www1.agenziaentrate.gov.it/servizi/geo...
2,Broken,Broken,blanksnekkecot34o
3,865.0,1240.0,https://www1.agenziaentrate.gov.it/servizi/geo...
4,NaN,NaN,https://www.google.it/
